**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/trajectories.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Use pyngrok on colab.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Let's do all of our imports here, too.
import numpy as np
import pydrake.all
import os
from IPython.display import display

In [ ]:
from ipywidgets import Textarea

from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.examples.manipulation_station import ManipulationStation
import pydrake.multibody.jupyter_widgets
import pydrake.systems.jupyter_widgets
from manipulation.utils import FindResource
from manipulation.scenarios import AddIiwa, AddWsg
from manipulation.jupyter_widgets import MakePoseSlidersThatPublishOnCallback

def AddShape(plant, shape, name, mass=1, mu=1, color=[.5, .5, .9, 1.0]):
    instance = plant.AddModelInstance(name)
    if isinstance(shape, pydrake.geometry.Box):
        inertia = pydrake.multibody.tree.UnitInertia.SolidBox(shape.width(), shape.depth(), shape.height())
    else:
        raise RunTimeError(f"need to add the unit inertia pair for shapes of type {shape}")
    body = plant.AddRigidBody(name, instance, pydrake.multibody.tree.SpatialInertia(mass=mass, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=inertia))
    if plant.geometry_source_is_registered():
        if isinstance(shape, pydrake.geometry.Box):
            plant.RegisterCollisionGeometry(
                body, RigidTransform(), pydrake.geometry.Box(shape.width()-0.001, shape.depth()-0.001, shape.height()-0.001), 
                name, pydrake.multibody.plant.CoulombFriction(mu, mu))
            i=0
            for x in [-shape.width() / 2.0, shape.width() / 2.0]:
                for y in [-shape.depth() / 2.0, shape.depth() / 2.0]:
                    for z in [-shape.height() / 2.0, shape.height() / 2.0]:
                        plant.RegisterCollisionGeometry(body, RigidTransform([x, y, z]), pydrake.geometry.Sphere(radius=1e-7), 
                                                        f"contact_sphere{i}", pydrake.multibody.plant.CoulombFriction(mu, mu))
                        i += 1
        else:
            plant.RegisterCollisionGeometry(body, RigidTransform(), shape, name, pydrake.multibody.plant.CoulombFriction(mu, mu))

        plant.RegisterVisualGeometry(body, RigidTransform(), shape, name, color)

    return instance



# Interactive inverse kinematics

This first cell gives us an interface that is very similar to the differential IK teleop interface that we used before.  See if you can spot any differences.

In [ ]:
def teleop_inverse_kinematics():
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa)
    plant.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)

    console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

    def my_callback(context, pose):
        ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(gripper_frame, [0, 0, 0], plant.world_frame(), pose.translation(), pose.translation())
        ik.AddOrientationConstraint(gripper_frame, RotationMatrix(), plant.world_frame(), pose.rotation(), 0.0)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.SetInitialGuess(q, q0)
        result = pydrake.solvers.mathematicalprogram.Solve(ik.prog())
        X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
        if not result.is_success():
            console.value += "IK failed\n"

    visualizer.load()
    X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
    MakePoseSlidersThatPublishOnCallback(visualizer, context, my_callback, value=X_WG)
    display(console)

teleop_inverse_kinematics()


This one has a collision to avoid.  Try moving it in negative y.

In [ ]:
def teleop_inverse_kinematics():
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa)
    box = AddShape(plant, pydrake.geometry.Box(0.2, 0.2, 0.2), "box")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("box", box), RigidTransform([0.2, 0.2, 0.5]))
    plant.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)

    console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

    def my_callback(context, pose):
        ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(gripper_frame, [0, 0, 0], plant.world_frame(), pose.translation(), pose.translation())
        ik.AddOrientationConstraint(gripper_frame, RotationMatrix(), plant.world_frame(), pose.rotation(), 0.0)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.SetInitialGuess(q, q0)
        result = pydrake.solvers.mathematicalprogram.Solve(ik.prog())
        X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
        if not result.is_success():
            console.value += "IK failed\n"

    visualizer.load()
    X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
    s = MakePoseSlidersThatPublishOnCallback(visualizer, context, my_callback, value=X_WG)
    s[5].value = 0.25  # set the initial z lower, to make the interaction interesting.
    display(console)

teleop_inverse_kinematics()
